In [24]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import numpy as np
from matplotlib.cm import ScalarMappable
from datetime import datetime, timedelta
import pandas as pd
from scipy.interpolate import griddata

# Define the start and end dates
start_date = datetime(2019, 6, 26)
end_date = datetime(2019, 6, 27)

# Define the directory to save the plots
save_directory = '/Users/poopy/Library/CloudStorage/OneDrive-UniversityofSouthernCalifornia/PARAGON_ms/PyRATES/'

# Define the extent of the map
map_extent = [-180, -134, 18, 35]  # [min_lon, max_lon, min_lat, max_lat]

# Define the grid for interpolation
lon_grid = np.linspace(map_extent[0], map_extent[1], 1000)
lat_grid = np.linspace(map_extent[2], map_extent[3], 1000)
lon_mesh, lat_mesh = np.meshgrid(lon_grid, lat_grid)

# Loop through each date
current_date = start_date
while current_date <= end_date:
    # Create the file name for the mimi data
    file_name_mimi = f'/Users/poopy/Library/CloudStorage/OneDrive-UniversityofSouthernCalifornia/PARAGON_ms/py/data/mimi_out/{current_date.strftime("%Y")}_sol_filtered/mimi_{current_date.strftime("%Y_%m_%d")}.csv'
    
    try:
        # Open the CSV file for the current date
        mimi = pd.read_csv(file_name_mimi)
    except FileNotFoundError:
        # Skip this date if the file does not exist
        print(f'File not found for date: {current_date.strftime("%Y-%m-%d")}')
        current_date += timedelta(days=1)
        continue

    # Initialize the data required to plot
    mimi_latitude = mimi['Latitude'].values
    mimi_longitude = mimi['Longitude'].values
    mimi_data = mimi['FESOLALL_Data'].values

    # Interpolate the data to a regular grid
    interpolated_data = griddata(
        (mimi_longitude, mimi_latitude), mimi_data, (lon_mesh, lat_mesh), method='linear'
    )

    # Create a figure and axis with Cartopy projection
    fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})
    ax.set_extent(map_extent)
    
    # Add background map and coastlines
    ax.add_feature(cfeature.LAND, edgecolor='black')
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.add_feature(cfeature.OCEAN)
    ax.add_feature(cfeature.LAKES, edgecolor='black')
    ax.add_feature(cfeature.RIVERS)

    # Add gridlines
    gl = ax.gridlines(draw_labels=True, linestyle='--', color='gray')
    gl.top_labels = False
    gl.right_labels = False
    gl.xformatter = LongitudeFormatter()
    gl.yformatter = LatitudeFormatter()

    # Define colormap normalization
    vmin = 0
    vmax = 2.0e-13
    sm = ScalarMappable(cmap='gist_heat_r', norm=plt.Normalize(vmin=vmin, vmax=vmax))

    # Create the colormesh plot for the interpolated data
    pcm = ax.pcolormesh(lon_mesh, lat_mesh, interpolated_data, cmap='gist_heat_r', vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree())

    # Overlay the scatter plot for the original data points
    sc = ax.scatter(mimi_longitude, mimi_latitude, c=mimi_data, cmap='gist_heat_r', vmin=vmin, vmax=vmax, s=10, edgecolor='k', transform=ccrs.PlateCarree())

    # Add the title
    ax.set_title(f'FESOLALL for {current_date.strftime("%Y-%m-%d")}')
    
    # Create colorbar by specifying the axes
    cax = fig.add_axes([1.0, 0.15, 0.02, 0.7])  # Adjust the position as needed
    cb = plt.colorbar(pcm, cax=cax, label='soluble Fe deposition flux [kg m$^{-2}$ s$^{-1}$]')

    # Save the figure with colorbar to the right
    save_filename = f'{save_directory}fesol_{current_date.strftime("%Y%m%d")}.jpg'
    plt.savefig(save_filename, dpi=300, bbox_inches='tight')
    plt.close()

    # Move to the next date
    current_date += timedelta(days=1)
